`01-tutorial.ipynb`

```sh
pip install langchain langchain-openai
```

## Basic Tutorial

In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
model = ChatOpenAI(model="gpt-3.5-turbo-1106")

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into korean"),
    HumanMessage(content="hi!"),
]

result = model.invoke(messages)

AIMessage(content='안녕하세요!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 21, 'total_tokens': 27}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_44132a4de3', 'finish_reason': 'stop', 'logprobs': None}, id='run-2024a1d4-93d1-483f-89aa-44c03fc9dc4c-0', usage_metadata={'input_tokens': 21, 'output_tokens': 6, 'total_tokens': 27})

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'안녕하세요!'

In [9]:
chain = model | parser
chain.invoke(messages)

'안녕하세요!'

In [15]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hawaian pizza and americano"})

result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hawaian pizza and americano')]

In [16]:
chain = prompt_template | model | parser
chain.invoke({"language": "italian", "text": "hawaian pizza and americano"})

'pizza hawaiana e americano'

## Chatbot Tutorials

In [18]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm neo")])

AIMessage(content='Hello, Neo! Nice to meet you. What can I help you with today?', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 12, 'total_tokens': 29}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_44132a4de3', 'finish_reason': 'stop', 'logprobs': None}, id='run-cee5a3e7-7e2f-4ceb-a944-350218adb317-0', usage_metadata={'input_tokens': 12, 'output_tokens': 17, 'total_tokens': 29})

In [19]:
model.invoke([HumanMessage(content="What's my name")])

AIMessage(content="I'm sorry, I don't know your name.", response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_44132a4de3', 'finish_reason': 'stop', 'logprobs': None}, id='run-d2bbb623-323e-41da-aeb2-6d02aaf7dafc-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22})

In [20]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Neo"),
        AIMessage(content="Hello Neo! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Neo.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_0ccba42292', 'finish_reason': 'stop', 'logprobs': None}, id='run-7ae571ca-1b63-41ed-8c8d-83165ae6aea4-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})

```sh
pip install langchain_community
```

In [21]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [26]:
config = {"configurable": {"session_id": "abc2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Neo")],
    config=config,
)

response.content

"Hi Neo! It's nice to meet you. How can I assist you today?"

In [28]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Neo.'

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello, Bob! How can I assist you today?'

In [30]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hello, Jim! How can I assist you today?'

In [31]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim. How can I assist you today, Jim?'

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [33]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [36]:
config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [37]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Tu nombre es Todd.'

In [38]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [39]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't know your name. How can I assist you today?"

In [40]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2?"'